In [1]:
!pip install tweepy
!pip install requests
!pip install requests-oauthlib
!pip install searchtweets
!pip install matplotlib
!pip install seaborn

In [2]:
!pip install --upgrade pip

In [3]:
#한글 정규화용
!pip install JPype1-1.1.2-cp38-cp38-win_amd64.whl

In [4]:
#한글 정규화 라이브러리
!pip install konlpy

In [5]:
from konlpy.tag import Kkma, Hannanum, Komoran, Mecab, Okt
from konlpy.utils import pprint

kkma = Kkma()

text = u'설치 중 오류 발생, 실행이 잘 되길 바랍니다. '

print(kkma.morphs(text))
print(kkma.nouns(text))
print(kkma.pos(text))
print(kkma.sentences(text))

['설치', '중', '오류', '발생', ',', '실행', '이', '잘', '되', '기', '를', '바라', 'ㅂ니다', '.']
['설치', '중', '오류', '발생', '실행']
[('설치', 'NNG'), ('중', 'NNB'), ('오류', 'NNG'), ('발생', 'NNG'), (',', 'SP'), ('실행', 'NNG'), ('이', 'JKS'), ('잘', 'MAG'), ('되', 'VV'), ('기', 'ETN'), ('를', 'JKO'), ('바라', 'VV'), ('ㅂ니다', 'EFN'), ('.', 'SF')]
['설치 중 오류 발생, 실행이 잘 되길 바랍니다.']


In [6]:
%matplotlib inline

import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [7]:
client = tweepy.Client(bearer_token='****', 
                       consumer_key='****', 
                       consumer_secret='****', 
                       access_token='****', 
                       access_token_secret='****')


In [8]:
def searchTweets(client, query, max_results):
    tweets = client.search_recent_tweets(query=query, max_results=max_results)
    #tweets = client.search_all_tweets(query=query, max_results=max_results)
    tweet_data =  tweets.data
    results = []
    
    if not tweet_data is None and len(tweet_data) > 0:
        for tweet in tweet_data:
            #obj = {}
            #obj['id'] = tweet.id
            #obj['text'] = tweet.text
            obj = tweet['text']
            #obj['date'] = tweet['created_at']
            #print(tweet['created_at'])
            if(obj[0] == 'R' and obj[1] == 'T'):
                continue
            results.append(obj) #obj-1 하면 맨 뒤는 잘리나?
    return results

def searchTweetsEx(client, query, max_results, start, end):#트위터 데이터 유형 확인?
    tweets = client.search_recent_tweets(query=query, max_results=max_results, start_time=start, end_time=end)
    #tweets = client.search_all_tweets(query=query, max_results=max_results)
    tweet_data =  tweets.data
    results = []
    
    '''if not tweet_data is None and len(tweet_data) > 0:
        for tweet in tweet_data:
            #obj = {}
            #obj['id'] = tweet.id
            #obj['text'] = tweet.text
            obj = tweet['text']
            #obj['date'] = tweet['created_at']
            #print(tweet['created_at'])
            if(obj[0] == 'R' and obj[1] == 'T'):
                continue
            results.append(obj)
    return results'''
    return tweets

def returnSearchTweetList(query, max_results):
    client = getClient()
    tweets = searchTweets(client, query, max_results)
    objs = []
    if len(tweets) > 0:
        for tweet in tweets:
            twt = getTweet(client, tweet['id'])
            obj = {}
            obj['text'] = tweet['text']
            obj['username'] = twt.includes['users'][0].username
            obj['id'] = tweet['id']
            obj['url'] = 'https://twitter.com/{}/status/{}'.format(twt.includes['users'][0].username, tweet['id'])
            objs.append(obj)
    return objs


In [23]:
#12시간 간격 크롤링 max로 뽑으면 1400개
start_date = ['2023-03-22T12:45:00Z', '2023-03-23T12:45:00Z'
                , '2023-03-24T12:45:00Z', '2023-03-25T12:45:00Z', '2023-03-26T12:45:00Z', '2023-03-27T12:45:00Z']
end_date = ['2023-03-22T23:45:00Z', '2023-03-23T23:45:00Z', '2023-03-24T23:45:00Z', '2023-03-25T23:45:00Z'
           , '2023-03-26T23:45:00Z', '2023-03-27T23:45:00Z']

In [24]:
with open('samsung_twit_list_ex.txt', 'a', encoding='utf-8') as f:
 
    for i in range(len(start_date)):
        tweets = searchTweetsEx(client, '삼성전자', 100, start_date[i], end_date[i])
        tweets_data = tweets.data
        for t in tweets_data:
            data = t['text']
            if(data[0] == 'R' and data[1] == 'T'):
                continue
            f.write(data)
            f.write('\n')
    
f.close()

In [ ]:
#이후로 저장된 데이터를 가공하는 과정

In [25]:
import re

def apply_regular_expression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

with open('samsung_twit_list_ex.txt', 'r', encoding = 'utf-8') as f:
    list = []
    not_split = []
    for r in f:
        not_split.append(r)
        row = r.split() #띄어쓰기 지우기
        for word in row:
            list.append(apply_regular_expression(word)) #마침표 삭제

f.close()
print(list)

['삼성전자', '그랑데', '통버블', '세탁기', '', '', '방문설치', '', '파트너스', '활동을', '통해', '일정액의', '수수료를', '제공', '받을수있음', '', '어메이징', '탕수육', '횡령한', '삼성전자', '', '삼성전자', '폭풍공감', '삼성전자', '전자동', '워블', '세탁기', '화이트', '', '', '방문설치', '', '파트너스', '활동을', '통해', '일정액의', '수수료를', '제공', '받을수있음', '', '어메이징', '탕수육', '횡령한', '삼성전자', '', '어메이징', '탕수육', '횡령한', '삼성전자', '', '삼성전자하이닉스', '', '정부', '발표에도', '', '리스크', '해소', '안', '돼', '미세공정', '전환', '못', '하면', '계륵', '신세', '', '삼성전자와', '하이닉스는', '중국', '공장에', '조원', '조원씩을', '투자했다', '미국의', '반도체', '제조', '장비', '수출금지', '조치로', '중국', '반도체', '기업의', '기술', '수준은', '당분간', '제자리걸음을', '할', '수밖에', '없다', '중국이', '필요로', '하는', '첨단', '반도체는', '한국', '기업에', '의존할', '수', '밖에', '없는', '것이다', '조선일보', '미국은', '삼성전자', '대만', '', '등을', '끌어들여', '미국', '내', '반도체', '생산', '라인을', '대거', '신설토록', '유도하고', '있다', '미', '정부는', '국립', '반도체', '기술센터를', '만들어', '차세대', '기술을', '개발하고', '차세대', '노광', '장비에서', '신소재', '개발', '양자', '정보통신까지', '향후', '반도체', '산업을', '이끌', '새로운', '표준을', '만들려는', '계획을', '갖고', '있다', '미친', '삼성전자의', '기술력', '', '삼성전자', '그랑데', 

In [26]:
new_list = []

for i in range(0, len(list)):
    if(list[i] != ''):
        new_list.append(list[i])
        
print(new_list)

['삼성전자', '그랑데', '통버블', '세탁기', '방문설치', '파트너스', '활동을', '통해', '일정액의', '수수료를', '제공', '받을수있음', '어메이징', '탕수육', '횡령한', '삼성전자', '삼성전자', '폭풍공감', '삼성전자', '전자동', '워블', '세탁기', '화이트', '방문설치', '파트너스', '활동을', '통해', '일정액의', '수수료를', '제공', '받을수있음', '어메이징', '탕수육', '횡령한', '삼성전자', '어메이징', '탕수육', '횡령한', '삼성전자', '삼성전자하이닉스', '정부', '발표에도', '리스크', '해소', '안', '돼', '미세공정', '전환', '못', '하면', '계륵', '신세', '삼성전자와', '하이닉스는', '중국', '공장에', '조원', '조원씩을', '투자했다', '미국의', '반도체', '제조', '장비', '수출금지', '조치로', '중국', '반도체', '기업의', '기술', '수준은', '당분간', '제자리걸음을', '할', '수밖에', '없다', '중국이', '필요로', '하는', '첨단', '반도체는', '한국', '기업에', '의존할', '수', '밖에', '없는', '것이다', '조선일보', '미국은', '삼성전자', '대만', '등을', '끌어들여', '미국', '내', '반도체', '생산', '라인을', '대거', '신설토록', '유도하고', '있다', '미', '정부는', '국립', '반도체', '기술센터를', '만들어', '차세대', '기술을', '개발하고', '차세대', '노광', '장비에서', '신소재', '개발', '양자', '정보통신까지', '향후', '반도체', '산업을', '이끌', '새로운', '표준을', '만들려는', '계획을', '갖고', '있다', '미친', '삼성전자의', '기술력', '삼성전자', '그랑데', '통버블', '세탁기', '방문설치', '파트너스', '활동을', '통해', '일정액의', '수수료를', '제공',

In [28]:
from konlpy.tag import Okt
from collections import Counter

counter = Counter(new_list)
counter.most_common(10)

[('삼성전자', 309),
 ('일정액의', 127),
 ('수수료를', 127),
 ('파트너스', 99),
 ('제공', 91),
 ('수', 77),
 ('갤럭시', 76),
 ('통해', 75),
 ('활동을', 73),
 ('받을수있음', 70)]

In [33]:
my_stopwords = ['파트너스', '수수료를', '일정액의', '제공', '활동', '받을수있음']

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 정규 표현식 처리
    result = hangul.sub('', text)
    okt = Okt()  # 형태소 추출
    nouns = okt.nouns(result)
    nouns = [x for x in nouns if len(x) > 1]  # 한글자 키워드 제거
    nouns = [x for x in nouns if x not in my_stopwords]  # 불용어 제거
    return nouns

vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
bow_vect = vect.fit_transform(not_split)
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)

In [35]:
# "단어" - "총 등장 횟수" Matching

word_count_dict = dict(zip(word_list, count_list))
word_count_dict

{'가격': 14,
 '가기': 1,
 '가망': 1,
 '가방': 1,
 '가성': 1,
 '가장': 3,
 '가전': 8,
 '가죽': 1,
 '가치투자': 20,
 '가카': 2,
 '간다': 1,
 '간섭': 1,
 '간첩': 1,
 '감산': 2,
 '강간': 8,
 '강남역': 2,
 '강도': 1,
 '강소': 1,
 '강원도': 9,
 '개막': 1,
 '개발': 4,
 '개새끼': 1,
 '개인': 3,
 '개입': 1,
 '개통': 2,
 '갤럭시': 90,
 '갤럭시탭': 64,
 '거기': 1,
 '거르': 3,
 '거리': 9,
 '거치': 1,
 '건강': 3,
 '걸이': 5,
 '검색': 1,
 '검토': 2,
 '게시': 1,
 '격려': 1,
 '격차': 2,
 '결과': 2,
 '결정': 2,
 '경기': 1,
 '경북': 1,
 '경영': 1,
 '경쟁': 1,
 '경쟁력': 2,
 '경제': 2,
 '경제부총리': 1,
 '경찰': 1,
 '경향신문': 1,
 '계륵': 2,
 '계약': 3,
 '계좌': 7,
 '계획': 4,
 '고객': 1,
 '고대': 1,
 '고소': 2,
 '고소장': 1,
 '고액': 2,
 '고용': 4,
 '고위': 4,
 '고함': 1,
 '곡면': 1,
 '골대': 1,
 '골드': 1,
 '곰돌이': 1,
 '공감': 2,
 '공기': 2,
 '공기청정기': 9,
 '공략': 2,
 '공매도': 20,
 '공시': 2,
 '공안': 1,
 '공예': 1,
 '공작': 8,
 '공장': 3,
 '공정': 3,
 '공주시': 1,
 '과기원': 4,
 '과정': 1,
 '과학기술원': 2,
 '관리': 1,
 '광고': 1,
 '광주': 1,
 '광주시': 2,
 '교육': 1,
 '교통사고': 1,
 '구독': 1,
 '구매': 32,
 '구영': 5,
 '구입': 3,
 '구조조정': 1,
 '국내': 7,
 '국립': 2,
 '국민': 3,
 '국제': 2,
 '군산시': 1,
 '권

In [36]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

print(tf_idf_vect.shape)

(1066, 889)


In [39]:
vect.vocabulary_

{'삼성': 360,
 '전자': 647,
 '버블': 298,
 '세탁기': 404,
 '방문': 293,
 '설치': 392,
 '파트너': 803,
 '통해': 793,
 '정액': 658,
 '수수료': 421,
 '어메이징': 488,
 '탕수육': 777,
 '횡령': 886,
 '폭풍': 822,
 '공감': 65,
 '자동': 615,
 '워블': 540,
 '화이트': 875,
 '하이닉스': 843,
 '정부': 657,
 '발표': 291,
 '리스크': 223,
 '해소': 856,
 '미세': 272,
 '공정': 75,
 '전환': 649,
 '계륵': 49,
 '신세': 458,
 '중국': 682,
 '공장': 74,
 '조원': 671,
 '투자': 794,
 '미국': 269,
 '반도체': 284,
 '제조': 665,
 '장비': 625,
 '수출': 427,
 '금지': 113,
 '조치': 672,
 '기업': 123,
 '기술': 120,
 '수준': 426,
 '당분간': 174,
 '제자리걸음': 664,
 '필요': 835,
 '첨단': 719,
 '한국': 849,
 '의존': 564,
 '조선일보': 670,
 '대만': 179,
 '생산': 375,
 '라인': 212,
 '대거': 175,
 '신설': 457,
 '유도': 553,
 '국립': 92,
 '센터': 407,
 '차세대': 706,
 '개발': 20,
 '노광': 154,
 '신소재': 459,
 '양자': 486,
 '정보': 656,
 '통신': 789,
 '향후': 862,
 '산업': 358,
 '표준': 824,
 '계획': 52,
 '블루': 329,
 '스카이': 434,
 '공기청정기': 67,
 '모델': 252,
 '명품': 250,
 '포스팅': 815,
 '쿠팡': 768,
 '일환': 604,
 '주식': 678,
 '차트': 708,
 '분석': 321,
 '기술적분석': 121,
 '참고': 711,
 '성전': 39

In [ ]:
from sklearn.model_selection import train_test_split

#random_idx = positive_random_idx + negative_random_idx
x = tf_idf_vect
y = 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)